In [1]:
import os
import numpy as np
import pandas as pd
import PIL.Image as Image

import torch 
import torch.nn as nn
import torch.functional as F

from torchvision.transforms import Compose, ToTensor, Resize

from collections import namedtuple


In [2]:
test_image = os.path.join('..','data','down-stream','binary','Normal','1.jpg')

In [3]:
class ConvBlock(nn.Module):
    def __init__(self,**kwargs):
        super(ConvBlock,self).__init__()
        
        self.block = nn.Sequential(nn.Conv2d(**kwargs),
                                   nn.BatchNorm2d(kwargs['out_channels']))
    
    def forward(self,x):
        x = self.block(x)
        return x

In [4]:
test_image = Image.open(test_image)

In [5]:
transforms = Compose([Resize((224,224)),
    ToTensor()
    ])

In [6]:
test_image = transforms(test_image).unsqueeze(0)
test_image.shape

torch.Size([1, 3, 224, 224])

In [7]:
test = ConvBlock(in_channels=3,out_channels=64,kernel_size=3, padding= 1)

In [8]:
test(test_image).shape

torch.Size([1, 64, 224, 224])

In [9]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self,in_channels,out_channels,stride = 1,downsample = False):
        super(BasicBlock,self).__init__()
        
        self.conv1 = ConvBlock(in_channels= in_channels, out_channels= out_channels, stride= stride,
                                kernel_size= 3, padding= 1, bias= False)
        
        self.conv2 = ConvBlock(in_channels= out_channels, out_channels= out_channels, stride= 1,
                                kernel_size = 3, padding= 1, bias= False)
        
        self.relu = nn.ReLU(inplace= True)
        
        if downsample:
            self.downsample = ConvBlock(in_channels= in_channels, out_channels= out_channels,
                                        kernel_size= 1, stride= 2, bias= False)
        else:
            self.downsample = None
            
        
    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
            
        if self.downsample != None:
            identity = self.downsample(identity)
        x += identity
        x = self.relu(x)
            
        return x

In [10]:
test = BasicBlock(in_channels= 3, out_channels= 64, stride= 2, downsample= True)

In [11]:
test(test_image).shape

torch.Size([1, 64, 112, 112])

In [12]:
class Bottleneck(nn.Module):
    
    expansion = 4
    def __init__(self,in_channels, out_channels, stride= 1, downsample= False):
        super(Bottleneck,self).__init__()
        
        self.conv1 = ConvBlock(in_channels= in_channels, out_channels= out_channels, stride= 1,
                               kernel_size= 1, bias= False)
        
        self.conv2 = ConvBlock(in_channels= out_channels, out_channels= out_channels, stride= stride,
                               kernel_size= 3, padding= 1, bias= False)
        
        self.conv3 = ConvBlock(in_channels= out_channels, out_channels= out_channels * self.expansion, 
                               stride= 1, kernel_size= 1, bias= False)
        
        self.relu = nn.ReLU(inplace= True)
        
        
        if downsample:
            self.downsample = ConvBlock(in_channels= in_channels, out_channels= out_channels * self.expansion,
                                        kernel_size=1, stride= stride, bias= False)
        else:
            self.downsample = None
            
    
    def forward(self,x):
        identity = x
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        
        if self.downsample != None:
            identity = self.downsample(identity)
            
        x += identity
        x = self.relu(x)
        
        return x
    

In [13]:
test = Bottleneck(in_channels= 3, out_channels= 64, stride= 2, downsample= True)

In [14]:
test(test_image).shape

torch.Size([1, 256, 112, 112])

In [19]:
class ResNet(nn.Module):
    def __init__(self,config, output_dim= 10, clf= True,image_channels=3):
        super(ResNet, self).__init__()
    
        block, n_blocks, channels = config
        self.image_channels= image_channels
        self.output_dim = output_dim
        self.clf = clf
        self.in_channels = channels[0]
    
        assert len(n_blocks) == len(channels) == 4
    
        self.conv1 = ConvBlock(in_channels= self.image_channels, out_channels= self.in_channels,kernel_size= 7, stride=2,
                           padding= 3, bias= False)
        self.relu = nn.ReLU(inplace= True)
        self.max_pool = nn.MaxPool2d(kernel_size= 3, stride= 2, padding=1)
        
    
        self.conv2_x = self._make_layer(block, n_blocks[0],channels[0])
        self.conv3_x = self._make_layer(block, n_blocks[1],channels[1], stride= 2)
        self.conv4_x = self._make_layer(block, n_blocks[2],channels[2], stride= 2)
        self.conv5_x = self._make_layer(block, n_blocks[3],channels[3], stride= 2)
    
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(self.in_channels, self.output_dim)
    
    
    
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avgpool(x)
        
        #if self.clf:
            #x = x.view(x.shape[0], -1)
            #x = self.fc(x)
        
        return x
    
    
    
    def _make_layer(self,block, n_blocks, channels, stride= 1):
        
        layers = []
        if self.in_channels != block.expansion * channels:
            downsample = True
        else:
            downsample = False
            
        layers.append(block(self.in_channels,channels,stride,downsample))
        
        for i in range(1,n_blocks):
            layers.append(block(block.expansion * channels, channels))
            
        self.in_channels = block.expansion * channels
        
        return nn.Sequential(*layers)
    

In [20]:
ResNetConfig = namedtuple('ResNetConfig', ['block', 'n_blocks', 'channels'])

In [21]:
resnet18_config = ResNetConfig(block = BasicBlock,
                               n_blocks = [2,2,2,2],
                               channels = [64, 128, 256, 512])

resnet34_config = ResNetConfig(block = BasicBlock,
                               n_blocks = [3,4,6,3],
                               channels = [64, 128, 256, 512])

resnet50_config = ResNetConfig(block = Bottleneck,
                               n_blocks = [3, 4, 6, 3],
                               channels = [64, 128, 256, 512])

resnet101_config = ResNetConfig(block = Bottleneck,
                                n_blocks = [3, 4, 23, 3],
                                channels = [64, 128, 256, 512])

resnet152_config = ResNetConfig(block = Bottleneck,
                                n_blocks = [3, 8, 36, 3],
                                channels = [64, 128, 256, 512])

In [22]:
model= ResNet(resnet50_config, output_dim=1000, image_channels=3, clf =True)
model(test_image).shape

torch.Size([1, 2048, 1, 1])

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 25,557,032 trainable parameters


In [46]:
from torchvision.models import resnet50

In [47]:
model = resnet50()

In [48]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 25,557,032 trainable parameters
